# Wissensaggregator Mittelalter und frühe Neuzeit

## Strukturdaten einlesen/aktualisieren

Inhalt

- [Länder einlesen](#Länder-einlesen)
- [Verwaltungsebenen einlesen](#Verwaltungsebenen-einlesen)
- [Orte aus GeoNames einlesen](#Orte-aus-GeoNames-einlesen)
  - [Fremsprachliche Namen](#Fremdsprachliche-Namen)
  - [Deutsche Namen eintragen](#Deutsche-Namen-eintragen)

Die Funktionen, welche Daten laden und umwandeln sind im Modul `WiagDataSetup` enthalten. Dieses verwendet seinerseits folgende Module:

- MySQL
- DataFrames
- JSON
- Infiltrator

Diese werden am besten vorab direkt in einem Julia-Terminal installiert.
``` julia
cd("C:\\Users\\Georg\\Documents\\projekte\\WiagDataSetup.jl")
Pkg.activate(".")
Pkg.add("MySQL")
Pkg.add("DataFrames")
Pkg.add("JSON")
Pkg.add("Infiltrator")
```

Pfad zum Modul `WiagDataSetup`

In [5]:
wds_path="../.."

"../.."

In [6]:
cd(wds_path)

In [7]:
using Pkg

In [8]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


Nur für die Entwicklung des Moduls relevant.

In [9]:
using Revise

Modul laden

In [10]:
using WiagDataSetup; Wds=WiagDataSetup

WiagDataSetup

Verbinde die Datenbank.

Im Verlauf der Einleseschritte kann die folgende Fehlermeldung erscheinen:
*Commands out of sync; you can't run this command now*.
In diesem Fall diesen Befehl nochmal absetzen.

In [11]:
Wds.setDBWIAG(user="georg", db="wiag_in")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in")

Verzeichnis für Basisdaten, z.b. SKOS-Schemes, Länder

In [12]:
data_path="C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup"

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup"

## Länder einlesen
Die Quelle enthält im Wesentlichen Deutschland und seine Nachbarn. Die numerische ID ist der numerische ISO-Code.

Quelle: https://download.geonames.org/export/dump/countryInfo.txt

In [9]:
using CSV, MySQL, DataFrames

In [10]:
country_file=joinpath(data_path, "GeoNames", "countryInfo.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\countryInfo.txt"

In [11]:
df_country = CSV.read(country_file, DataFrame);

In [12]:
first(df_country, 5)

Row,ISO,ISO3,ISO-Numeric,fips,Country,Capital,Area(in sq km),Population,Continent,tld,CurrencyCode,CurrencyName,Phone,Postal Code Format,Postal Code Regex,Languages,geonameid,neighbours,EquivalentFipsCode
,String3,String3,Int64,String3?,String,String31?,Float64?,Int64,String3,String3?,String3?,String15?,String31?,String?,String?,String?,Int64?,String?,String3?
1,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,84000,EU,.ad,EUR,Euro,376,AD###,^(?:AD)*(\\d{3})$,ca,3041565,"ES,FR",missing
2,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,4975593,AS,.ae,AED,Dirham,971,missing,missing,"ar-AE,fa,en,hi,ur",290557,"SA,OM",missing
3,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,29121286,AS,.af,AFN,Afghani,93,missing,missing,"fa-AF,ps,uz-AF,tk",1149361,"TM,CN,IR,TJ,PK,UZ",missing
4,AG,ATG,28,AC,Antigua and Barbuda,St. John's,443.0,86754,NA,.ag,XCD,Dollar,+1-268,missing,missing,en-AG,3576396,missing,missing
5,AI,AIA,660,AV,Anguilla,The Valley,102.0,13254,NA,.ai,XCD,Dollar,+1-264,missing,missing,en-AI,3573511,missing,missing


In [13]:
df_country[end-5:end, [Symbol("ISO-Numeric"), :Country]]

Row,ISO-Numeric,Country
,Int64,String
1,175,Mayotte
2,710,South Africa
3,894,Zambia
4,716,Zimbabwe
5,891,Serbia and Montenegro
6,530,Netherlands Antilles


In [14]:
select_cols = [
    Symbol("ISO-Numeric") => :id,
    :Country => :name,
    :ISO => :country_code,
    :Capital => :capital
]

4-element Vector{Pair{Symbol, Symbol}}:
 Symbol("ISO-Numeric") => :id
              :Country => :name
                  :ISO => :country_code
              :Capital => :capital

In [15]:
df_country_db = select(df_country, select_cols...);

In [ ]:
Wds.filltable!("country", df_country_db, clear_table = true)

GeoNames-IDs ergänzen

Quelle: `wiag_bundeslaender_normdaten_lhofman.xls` übertragen in `country.csv`

In [ ]:
using MySQL, DataFrames, CSV

In [ ]:
cy_file=joinpath(data_path, "csv", "country.csv")

In [ ]:
df_cy=CSV.read(cy_file, DataFrame)

In [ ]:
sql="DROP TABLE IF EXISTS country_gn_id"
DBInterface.execute(Wds.dbwiag, sql);

In [ ]:
sql="CREATE TEMPORARY TABLE country_gn_id (" *
"id INT," *
"country_code VARCHAR(31)," *
"country_code_3 VARCHAR(31)," *
"geonames_id INT)"
DBInterface.execute(Wds.dbwiag, sql);

In [ ]:
Wds.filltable!("country_gn_id", df_cy[!, [:id, :country_code, :country_code_3, :geonames_id]])

In [ ]:
sql="UPDATE country AS cy, (SELECT id, country_code, country_code_3, geonames_id FROM country_gn_id) as gn " *
"SET cy.country_code_3 = gn.country_code_3, cy.geonames_id = gn.geonames_id " *
"WHERE cy.id = gn.id"
DBInterface.execute(Wds.dbwiag, sql);

externe IDs ergänzen  
2022-04-08: Es liegen nur externe IDs für Länder vor. Diese werden nicht übernommen; bzw. wieder gelöscht.

In [ ]:
cei_file=joinpath(data_path, "csv", "country_id_external.csv")

In [ ]:
df_cei=CSV.read(cei_file, DataFrame, types=[Int, String, String, String, Int, String]);

In [ ]:
size(df_cei)

In [ ]:
df_cei[1:7, :]

In [ ]:
Wds.filltable!("place_id_external", df_cei[!, [:geonames_id, :authority_id, :value]])

externe IDs für Bundesländer ergänzen

Quelle: Quelle: `wiag_bundeslaender_normdaten_lhofman.xls` übertragen in `country_level_1_id_external.csv`

In [ ]:
cei_l1_file=joinpath(data_path, "csv", "country_level_1_id_external.csv")

In [ ]:
df_cei_l1=CSV.read(cei_l1_file, DataFrame, types=[Int, String, String, String, Int, String]);

In [ ]:
df_cei_l1[1:7, :]

In [ ]:
Wds.filltable!("place_id_external", df_cei_l1[!, [:geonames_id, :authority_id, :value]])

Kontrolle

``` sql
SELECT name, country_code, authority_id, value 
FROM place_id_external AS pei
JOIN country_level_1 AS cl1 ON pei.geonames_id = cl1.geonames_id;
```

Es fehlen Hamburg, Bremen

## Verwaltungsebenen einlesen
Die Quelle enthält die oberste Verwaltungsebene nach dem Land selbst, also für Deutschland die Bundesländer, für Frankreich die Regionen. Die Quelle umfasst alle Daten in GeoNames und wird daher gefiltert.

In [ ]:
using DataFrames; using CSV

In [ ]:
cl1_file=joinpath(data_path, "Geonames", "admin1CodesASCII.txt")

In [ ]:
df_cl1=CSV.read(cl1_file, DataFrame, header=["code", "name", "ascii_name", "geonames_id"]);

In [ ]:
first(df_cl1, 7)

Länder Code extrahieren

In [ ]:
get_country_code(code)=split(code::AbstractString, ".")[1]

In [ ]:
df_cl1[!, :country_code] .= get_country_code.(df_cl1[!, :code]);

In [ ]:
first(df_cl1, 7)

Code der Verwaltungsebene extrahieren

In [ ]:
get_admin1_code(code)=split(code::AbstractString, ".")[2]

In [ ]:
df_cl1[!, :admin1_code] .= get_admin1_code.(df_cl1[!, :code]);

Länder auslesen

In [ ]:
using MySQL

In [ ]:
sql = "SELECT id as country_id, country_code FROM country " * 
"WHERE country_code in (SELECT distinct(country_code) FROM place)"
df_country_m = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [ ]:
delete!(df_country_m, 16)

In [ ]:
df_ccl1 = leftjoin(df_country_m, df_cl1, on = :country_code);

Index ergänzen

In [ ]:
df_ccl1[!, :id] .= 1:size(df_ccl1, 1);

In [ ]:
first(df_ccl1[!, [:id, :country_id, :country_code, :name]], 5)

In [ ]:
size(df_ccl1)

In [ ]:
import_cols = [:id, :country_id, :country_code, :admin1_code, :name, :ascii_name, :geonames_id]

In [ ]:
names(df_ccl1)

In [ ]:
Wds.filltable!("country_level_1", select(df_ccl1, import_cols), clear_table=true)

## Orte aus GeoNames einlesen
Geonames stellt eine Sammlung aller Objekte (features) zur Verfügung. Alle Objekte eines Landes sind in jeweils einer Datei enthalten.
https://download.geonames.org/export/dump/

Lies Daten zu 
- Deutschland
- Nachbarländer
- Baltikum
- Kroatien
- Kaliningrad/Königsberg

Ländercodes: DE, NL, BE, FR, IT, CH, AT, DK, PL, LU, CZ, LI, LV, LT, HR, EE, RU

Pfad zu den Daten

In [1]:
gn_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames"

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames"

Wähle aus den Features in den Daten von GeoNames nur Orte (P) aus, sowie Klöster (MSTY) und Konvente (CVNT).
Siehe http://www.geonames.org/export/codes.html.

Filterfunktion definieren

In [2]:
function filter_places(feature_class, feature_code)
    return ((!ismissing(feature_class) && feature_class == "P")
            || (!ismissing(feature_code) && feature_code in ("MSTY", "CVNT")))
end

filter_places (generic function with 1 method)

Spaltennamen und Spaltentypen

Länder auslesen

In [3]:
using MySQL
using DataFrames
using CSV

In [13]:
df_country_m = DBInterface.execute(Wds.dbwiag, "SELECT id as country_id, country_code FROM country") |> DataFrame;

In [14]:
first(df_country_m, 5)

Row,country_id,country_code
,Int32,String?
1,0,XK
2,4,AF
3,8,AL
4,10,AQ
5,12,DZ


In [15]:
gn_header = ["geonames_id", "name", "ascii_name", "alternatenames",
             "latitude", "longitude", "feature_class", "feature_code",
             "country_code", "cc2", "admin1_code", "admin2_code", "admin3_code", "admin4_code",
             "population", "elevation", "dem", "timezone", "modification_date"];

gn_types = [Int, String, String, String, Float64, Float64,
            String, String, String, String,
            String, String, String, String,
            Int, Int, Int, String, String];

In [138]:
place_cols = [:id_in_source, :country_id, :name, :ascii_name, :latitude, :longitude, 
        :feature_class, :feature_code, :country_code, :cc2, :admin1_code,
        :population, :elevation, :dem, :timezone, :modification_date, :geonames_id, :place_type_id];

### Niederlande

In [ ]:
gn_filename = joinpath(gn_path, "NL", "NL.txt")

In [ ]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [ ]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [ ]:
size(gn_dff)

In [ ]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [ ]:
gn_dff[1:5, [:country_id, :name, :latitude, :longitude, :elevation, :dem, :population]]

In [ ]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

### Belgien

In [ ]:
gn_filename = joinpath(gn_path, "BE", "BE.txt")

In [ ]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [ ]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [ ]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

In [ ]:
size(gn_dff)

Spalte für die Länder_ID einfügen

In [ ]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [ ]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

### Frankreich

In [ ]:
gn_filename = joinpath(gn_path, "FR", "FR.txt")

In [ ]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [ ]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [ ]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

In [ ]:
size(gn_dff)

Spalte für die Länder_ID einfügen

In [ ]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [ ]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

### Italien

In [ ]:
gn_filename = joinpath(gn_path, "IT", "IT.txt")

In [ ]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [ ]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [ ]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

In [ ]:
size(gn_dff)

Spalte für die Länder_ID einfügen

In [ ]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [ ]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

### Schweiz

In [ ]:
gn_filename = joinpath(gn_path, "CH", "CH.txt")

In [ ]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [ ]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [ ]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

In [ ]:
size(gn_dff)

Spalte für die Länder_ID einfügen

In [ ]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [ ]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

### Österreich

In [ ]:
gn_filename = joinpath(gn_path, "AT", "AT.txt")

In [ ]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [ ]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [ ]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

In [ ]:
size(gn_dff)

Spalte für die Länder_ID einfügen

In [ ]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [ ]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

### Dänemark

In [ ]:
gn_filename = joinpath(gn_path, "DK", "DK.txt")

In [ ]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [ ]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [ ]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

In [ ]:
size(gn_dff)

Spalte für die Länder_ID einfügen

In [ ]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [ ]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

### Polen

In [ ]:
gn_filename = joinpath(gn_path, "PL", "PL.txt")

In [ ]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [ ]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [ ]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

In [ ]:
size(gn_dff)

Spalte für die Länder_ID einfügen

In [ ]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [ ]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

### Deutschland

In [19]:
gn_filename = joinpath(gn_path, "DE", "DE.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\DE\\DE.txt"

In [20]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [24]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [ ]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

In [25]:
size(gn_dff)

(79810, 19)

Spalte für die Länder_ID einfügen

In [ ]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [ ]:
gn_dff[1:5, :]

In [ ]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

Übernehme auch historische Orte (z.B. Abtei Corvey)

In [29]:
function filter_places_hist(feature_class, feature_code)
    return (!ismissing(feature_code) && feature_code in ("MSTY", "CVNT", "HSTS"))
end

filter_places_hist (generic function with 1 method)

In [46]:
gn_dff_hist = filter([:feature_class, :feature_code] => filter_places_hist, gn_df);

In [47]:
size(gn_dff_hist)

(267, 19)

In [48]:
gn_dff_hist = rightjoin(df_country_m, gn_dff_hist, on = :country_code);

In [49]:
insertcols!(
    gn_dff_hist,
    :id_in_source => gn_dff_hist.geonames_id,
);

In [55]:
gn_dff_hist[151:157, [:name, :geonames_id, :id_in_source]]

Row,name,geonames_id,id_in_source
,String,Int64,Int64
1,"Freiburg, ehemalige Dominikanerkirche",11428014,11428014
2,"Regensburg, Minoritenkirche",11428015,11428015
3,"Engelthal, Ev. Kirche, Klosterkirche Johannes d. T.",11428017,11428017
4,Abtei Sayn,11429093,11429093
5,Seeberg Observatory (1787-1934),11494732,11494732
6,Seehofer Wasserspielreservoir,11497107,11497107
7,Kloster Michelsberg,11497656,11497656


In [58]:
table = "place"
Wds.filltable!(table, gn_dff_hist[!, place_cols], clear_table = false)

┌ Info: Rows inserted: 267
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


267

### Lies auch Orte mit Feature Code A ein
Zum Teil wird in der Klosterdatenbank auch diese GeoNames-ID verwendet. Wir beschränken uns auf die Länder, wo das vorkommt.

In [127]:
country_code_list = ["DE", "AT", "CH", "NL", "BE", "PL"]

6-element Vector{String}:
 "DE"
 "AT"
 "CH"
 "NL"
 "BE"
 "PL"

In [128]:
n = 0
gn_df_list = []
for country_code in country_code_list
    @info country_code
    gn_filename = joinpath(gn_path, country_code, country_code * ".txt")
    gn_df_loop = CSV.read(gn_filename, DataFrame, header = gn_header, types = gn_types)
    push!(gn_df_list, gn_df_loop)    
    n += 1
end

┌ Info: DE
└ @ Main In[128]:4
┌ Info: AT
└ @ Main In[128]:4
┌ Info: CH
└ @ Main In[128]:4
┌ Info: NL
└ @ Main In[128]:4
┌ Info: BE
└ @ Main In[128]:4
┌ Info: PL
└ @ Main In[128]:4


In [129]:
gn_df = vcat(gn_df_list...);

In [106]:
function filter_places_A(feature_class, feature_code)
    return ((!ismissing(feature_class) && feature_class in ("A",))
            || (!ismissing(feature_code) && feature_code in ("MSTY", "CVNT")))
end

filter_places_A (generic function with 1 method)

In [143]:
gn_dff_A = filter([:feature_class, :feature_code] => filter_places_A, gn_df);

In [144]:
size(gn_dff_A)

(22657, 19)

Lies die Orte der Klosterdatenbank ein, die Feature-Code A haben, um zu sehen, ob die Orte gefunden werden.

In [109]:
path_mon_db_A = joinpath(
    "C:\\",
    "Users",
    "georg",
    "Documents",
    "projekte-doc",
    "WIAGweb2",
    "data",
    "Klosterort_Geonames_ID.csv"
)

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\Klosterort_Geonames_ID.csv"

In [110]:
mon_db_A = CSV.read(path_mon_db_A, DataFrame);

In [111]:
mon_db_A = unique(mon_db_A, :geonames_id);

In [112]:
size(mon_db_A)

(175, 3)

In [132]:
gn_dff_A_sel = innerjoin(gn_dff_A, mon_db_A, on = :geonames_id);

In [133]:
size(gn_dff_A_sel)

(167, 21)

In [134]:
mon_db_A_mg = antijoin(mon_db_A, gn_dff_A_sel, on = :geonames_id)

Row,place_name,gsn,geonames_id
,String,Int64,Int64
1,Almkerk,8156,275987
2,Insel Lützelau,10006,9609557
3,Mont Sainte-Odilie (Odilienberg),99993,11002262
4,Oestrich-Winkel,30268,3207534
5,Salenstein,4779,71
6,Snamensk (Wehlau),19968,462322
7,Sovetsk (Tilsit),19965,490068
8,Zierikzee,8681,274391


Snamensk und Sovetsk sind schon als (ausgewählte) Orte in Russland mit Feature Code P aufgenommen (siehe unten). Sechs der Orte sind händisch nachzupflegen (Tippfehler, anderer Feature Code)  
*2023-01-11*

Spalte für die Länder_ID einfügen; `id_in_source` ergänzen, `place_type_id` ergänzen

In [145]:
gn_dff_A = rightjoin(df_country_m, gn_dff_A, on = :country_code);

In [146]:
transform!(gn_dff_A, :geonames_id => ByRow(string) => :id_in_source);

In [147]:
place_type_id = 1 # geonames
insertcols!(gn_dff_A, :place_type_id => place_type_id);

In [148]:
size(gn_dff_A)

(22657, 22)

In [150]:
Wds.filltable!("place", select(gn_dff_A, place_cols), clear_table = false)

┌ Info: 10000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1193
┌ Info: 20000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1193
┌ Info: Rows inserted: 22657
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


22657

### Luxemburg

In [ ]:
gn_filename = joinpath(gn_path, "LU", "LU.txt")

In [ ]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [ ]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [ ]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

In [ ]:
size(gn_dff)

Spalte für die Länder_ID einfügen

In [ ]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [ ]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

### Tschechien

In [ ]:
gn_filename = joinpath(gn_path, "CZ", "CZ.txt")

In [ ]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [ ]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [ ]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

In [ ]:
size(gn_dff)

Spalte für die Länder_ID einfügen

In [ ]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [ ]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

### Liechtenstein

In [ ]:
gn_filename = joinpath(gn_path, "LI", "LI.txt")

In [ ]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [ ]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [ ]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

In [ ]:
size(gn_dff)

Spalte für die Länder_ID einfügen

In [ ]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [ ]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

### Estland

Die erste Zeile der Daten kann nicht gelesen werden. Sie wird manuell aus den Quelldaten gelöscht. Das ist unerheblich, weil es sich nicht um ein relevantes Feature handelt.

In [ ]:
gn_filename = joinpath(gn_path, "EE", "EE-x1.txt")

In [ ]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [ ]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [ ]:
size(gn_dff)

In [ ]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

In [ ]:
size(gn_dff)

Spalte für die Länder_ID einfügen

In [ ]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [ ]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

### Lettland

In [ ]:
gn_filename = joinpath(gn_path, "LV", "LV.txt")

In [ ]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [ ]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [ ]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

In [ ]:
size(gn_dff)

Spalte für die Länder_ID einfügen

In [ ]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [ ]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

### Litauen

In [ ]:
gn_filename = joinpath(gn_path, "LT", "LT.txt")

In [ ]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [ ]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [ ]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

In [ ]:
size(gn_dff)

Spalte für die Länder_ID einfügen

In [ ]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [ ]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

### Kroatien

In [ ]:
gn_filename = joinpath(gn_path, "HR", "HR.txt")

In [ ]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [ ]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [ ]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

In [ ]:
size(gn_dff)

Spalte für die Länder_ID einfügen

In [ ]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [ ]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

### Slovenien

In [60]:
gn_filename = joinpath(gn_path, "SI", "SI.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\SI\\SI.txt"

In [61]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [74]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [75]:
size(gn_dff)

(6659, 19)

Spalte für die Länder_ID einfügen

In [76]:
gn_dff = rightjoin(df_country_m, gn_dff, on = :country_code);

In [78]:
insertcols!(
    gn_dff,
    :id_in_source => gn_dff.geonames_id,
);

In [79]:
gn_dff[1:5, [:id_in_source, :geonames_id, :name, :country_code, :population]]

Row,id_in_source,geonames_id,name,country_code,population
,Int64,Int64,String,String,Int64
1,3186234,3186234,Rašica,SI,267
2,3186239,3186239,Mrše,SI,29
3,3186254,3186254,Žvirče,SI,125
4,3186269,3186269,Žvab,SI,89
5,3186270,3186270,Žužemberk,SI,1053


In [80]:
Wds.filltable!("place", gn_dff[!, place_cols], clear_table = false)

┌ Info: Rows inserted: 6659
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


6659

### Königsberg - Russland

In [69]:
gn_filename = joinpath(gn_path, "RU", "RU.txt")

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\RU\\RU.txt"

In [70]:
gn_df = CSV.read(gn_filename, DataFrame, header=gn_header, types=gn_types);

filtern nach Art des Features

In [71]:
gn_dff = filter([:feature_class, :feature_code] => filter_places, gn_df);

In [72]:
gn_dff[1:5, [:geonames_id, :name, :country_code, :population]]

Row,geonames_id,name,country_code,population
,Int64,String,String,Int64
1,451747,Zyabrikovo,RU,0
2,451748,Znamenka,RU,0
3,451749,Zhukovo,RU,0
4,451750,Zhitovo,RU,0
5,451751,Zhitnikovo,RU,0


In [ ]:
size(gn_dff)

Extrahiere Königsberg

In [ ]:
gn_dff_kb = filter(:name => isequal("Kaliningrad"), gn_dff)

Spalte für die Länder_ID einfügen

In [ ]:
gn_dff_kb = rightjoin(df_country_m, gn_dff_kb, on = :country_code);

In [ ]:
Wds.filltable!("place", gn_dff_kb[!, place_cols], clear_table = false)

In [ ]:
DBInterface.execute(Wds.dbwiag, "SELECT COUNT(*) FROM place") |> DataFrame

Extrahiere Znamensk, Sovetsk; ergänze um die neue Spalte `id_in_source` mit dem Inhalt von `geonames_id`.
*2023-01-11*

In [82]:
filter_ru2(geonames_id) = geonames_id in (462322, 490068)

filter_ru2 (generic function with 1 method)

In [92]:
gn_dff_ru2 = subset(gn_dff, :geonames_id => ByRow(filter_ru2));

Spalte für die Länder_ID einfügen, sowie `id_in_source` und `place_type_id`

In [93]:
gn_dff_ru2 = rightjoin(df_country_m, gn_dff_ru2, on = :country_code)

Row,country_id,country_code,geonames_id,name,ascii_name,alternatenames,latitude,longitude,feature_class,feature_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
,Int32?,String,Int64,String,String?,String?,Float64,Float64,String,String,String?,String?,String?,String?,String?,Int64,Int64?,Int64,String?,String
1,643,RU,462322,Znamensk,Znamensk,"Snamensk,Velau,Veluva,Vėluva,Wehlau,Znamensk,Znamenskas,Znamiensk,ci na mian si ke,znamynsk,Знаменск,زنامينسك,兹纳缅斯克",54.614,21.2272,P,PPL,missing,23,missing,missing,missing,4600,missing,7,Europe/Kaliningrad,2016-02-26
2,643,RU,490068,Sovetsk,Sovetsk,"Saveck,Sovetk,Sovets'k,Sovetsk,Sovetsk i Kaliningrad oblast,Sovetska,Sovetskas,Sovețk,Sovietsk,Sovjetsk,Sovyetsk,Sowetsk,Sowieck,Sowjetsk,Szovjetszk,Til'zit,Tilsa,Tilsit,Tilze,Tilžė,Til’zit,sobecheukeu,sovuietsuku,su wei ai ci ke,swbtsq,swftsk,swwtsk kalynyngrad,Савецк,Советск,Совєтськ,Совјетск,סובטסק,سوفتسك,سووتسک، کالینینگراد,ソヴィェツク,苏维埃茨克,소베츠크",55.0839,21.8785,P,PPLA2,missing,23,missing,missing,missing,43309,missing,13,Europe/Kaliningrad,2019-09-05


In [94]:
transform!(gn_dff_ru2, :geonames_id => ByRow(string) => :id_in_source)

Row,country_id,country_code,geonames_id,name,ascii_name,alternatenames,latitude,longitude,feature_class,feature_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date,id_in_source
,Int32?,String,Int64,String,String?,String?,Float64,Float64,String,String,String?,String?,String?,String?,String?,Int64,Int64?,Int64,String?,String,String
1,643,RU,462322,Znamensk,Znamensk,"Snamensk,Velau,Veluva,Vėluva,Wehlau,Znamensk,Znamenskas,Znamiensk,ci na mian si ke,znamynsk,Знаменск,زنامينسك,兹纳缅斯克",54.614,21.2272,P,PPL,missing,23,missing,missing,missing,4600,missing,7,Europe/Kaliningrad,2016-02-26,462322
2,643,RU,490068,Sovetsk,Sovetsk,"Saveck,Sovetk,Sovets'k,Sovetsk,Sovetsk i Kaliningrad oblast,Sovetska,Sovetskas,Sovețk,Sovietsk,Sovjetsk,Sovyetsk,Sowetsk,Sowieck,Sowjetsk,Szovjetszk,Til'zit,Tilsa,Tilsit,Tilze,Tilžė,Til’zit,sobecheukeu,sovuietsuku,su wei ai ci ke,swbtsq,swftsk,swwtsk kalynyngrad,Савецк,Советск,Совєтськ,Совјетск,סובטסק,سوفتسك,سووتسک، کالینینگراد,ソヴィェツク,苏维埃茨克,소베츠크",55.0839,21.8785,P,PPLA2,missing,23,missing,missing,missing,43309,missing,13,Europe/Kaliningrad,2019-09-05,490068


In [136]:
insertcols!(gn_dff_ru2, :place_type_id => 1);

In [95]:
Wds.filltable!("place", select(gn_dff_ru2, place_cols), clear_table = false)

┌ Info: Rows inserted: 2
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


2

In [ ]:
DBInterface.execute(Wds.dbwiag, "SELECT COUNT(*) FROM place") |> DataFrame

### Fremdsprachliche Namen

In [17]:
using CSV, DataFrames, MySQL

In [18]:
gnl_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames"

"C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\GeoNames\\alternatenames"

In [19]:
gnl_header = ["id", "geonames_id", "lang", "label", 
              "is_preferred", "isShort", "isColloquial", "is_historic", "from", "to"];

gnl_types = [Int, Int, String, String,
             Int, Int, Int, Int, String, String];

In [20]:
lang_codes = ["la", "fr", "cz", "de", "pl", "en", "nl", "it"];

In [21]:
filter_lang(lc) = !ismissing(lc) && lc in lang_codes

filter_lang (generic function with 1 method)

In [22]:
country_codes = ["DE", "NL", "BE", "FR", "IT", "CH", "AT", "DK",
    "PL", "LU", "CZ", "LI", "LV", "LT", "HR", "EE", "RU"];

### Schleife über die Länder

Orte einlesen, um nur relevante Namen zu übernehmen

In [287]:
sql = "SELECT id as place_id, name, geonames_id " *
      "FROM place WHERE place_type_id = 1"
p_df = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [290]:
size(p_df)

(391240, 3)

In [291]:
function labels_by_country(cc)
    gnl_filename = joinpath(gnl_path, cc, cc * ".txt")
    gnl_df = CSV.read(gnl_filename, DataFrame, header=gnl_header, types=gnl_types);
    gnl_df = filter(:lang => filter_lang, gnl_df);
    gnl_p_df = innerjoin(gnl_df, p_df, on = :geonames_id);
    @info cc
    n_cc = Wds.filltable!("place_label", select(gnl_p_df, Not([:isShort, :isColloquial, :from, :to, :name])))    
end
    

labels_by_country (generic function with 1 method)

In [294]:
labels_by_country.(country_codes)

┌ Info: DE
└ @ Main In[291]:6
┌ Info: Rows inserted: 4364
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1209
┌ Info: NL
└ @ Main In[291]:6
┌ Info: Rows inserted: 1848
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1209
┌ Info: BE
└ @ Main In[291]:6
┌ Info: Rows inserted: 862
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1209
┌ Info: FR
└ @ Main In[291]:6
┌ Info: 10000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1186
┌ Info: 20000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1186
┌ Info: 30000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1186
┌ Info: Rows inserted: 33108
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1209
┌ Info: IT
└ @ Main In[291]:6
┌ Info: 10000
└ @ WiagDataSetup

17-element Vector{Int64}:
  4364
  1848
   862
 33108
 32162
  6277
   728
   669
  3859
   203
  1240
    72
   529
   787
   481
   663
    11

### Deutsche Namen eintragen

Trage in `place` für Orte in Deutschland, Österreich und der Schweiz den deutschen Namen ein.

In [ ]:
using MySQL, DataFrames

In [ ]:
db_exec(sql) = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame

In [ ]:
sql = "SELECT label, p.name FROM place_label AS pl " *
"JOIN place AS p ON pl.geonames_id = p.geonames_id " *
"WHERE pl.lang = 'de' AND p.country_code = 'DE' " *
"AND pl.label <> p.name " *
"LIMIT 12"
df_name_udt = db_exec(sql)

In [ ]:
sql = "SELECT count(*) FROM place_label AS pl " *
"JOIN place AS p ON pl.geonames_id = p.geonames_id " *
"WHERE pl.lang = 'de' AND p.country_code = 'DE' " *
"AND pl.label <> p.name "
n = db_exec(sql)

Es scheint im Allgemeinen keine gute Idee zu sein, generell den deutschen Eintrag aus `place_label` zu übernehmen. Man wird einzelne Orte evtl. redaktionell bearbeiten müssen, indem man einen deutschen Namen in `place_label` als bevorzugten Namen auszeichnet.

## Neue Organisation der Orte
Unterscheide Orte nach ihren Quellen (analog zu Items) (Tabelle `place_type`)

### Tabelle `place_type`
über DbSchema

In [ ]:
out_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WiagDataSetup\\data_sql"

In [11]:
using DataFrames, Dates

In [12]:
df_place_type = DataFrame();

In [13]:
insertcols!(df_place_type,
    :id => [1],
    :name => ["Ort GN"],
    :note => ["Orte aus GeoNames"],
    :created_by => 7,
    :date_created => now(),
    :changed_by => 7,
    :date_changed => now(),
    :table_name => "place",
    :name_app => "place",
)

,id,name,note,created_by,date_created,changed_by
,Int64,String,String,Int64,DateTime,Int64
1,1,Ort GN,Orte aus GeoNames,7,2022-04-11T09:06:26.483,7


In [14]:
rec_place_ut = (
    id = 2,
    name = "Ort Utrecht",
    note = "Orte der Priester aus Utrecht",
    created_by = 7,
    date_created = now(),
    changed_by = 7,
    date_changed = now(),
    table_name = "place",
    name_app = "place_ut",
)

(id = 2, name = "Ort Utrecht", note = "Orte der Priester aus Utrecht", created_by = 7, date_created = DateTime("2022-04-11T09:06:29.865"), changed_by = 7, date_changed = DateTime("2022-04-11T09:06:29.865"), table_name = "place", name_app = "place_ut")

In [15]:
push!(df_place_type, rec_place_ut)

,id,name,note,created_by,date_created
,Int64,String,String,Int64,DateTime
1,1,Ort GN,Orte aus GeoNames,7,2022-04-11T09:06:26.483
2,2,Ort Utrecht,Orte der Priester aus Utrecht,7,2022-04-11T09:06:29.865


In [16]:
table_name = "place_type";
Wds.filltable!(table_name, df_place_type)

┌ Info: Rows inserted: 2
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1209


2

`place_type_id` nachtragen
```sql
UPDATE place SET place_type_id = 1;
```

`id_in_source` nachtragen
```sql
UPDATE place SET id_in_source = geonames_id;
```

`geonames_id` als Index ersetzen durch `place_id`

```sql
UPDATE place_label AS pll, (SELECT id, geonames_id FROM place) as p
SET pll.place_id = p.id
WHERE pll.geonames_id = p.geonames_id;
```

Einträge in `place_label` löschen, die sich auf Länder, Kantone, Bundesländer und also nicht auf Orte beziehen

```sql
DELETE FROM place_label WHERE place_id IS NULL;
```

In `place_label` die Namen aus der GeoNames Ortetabelle übernehmen, wie von bk vorgeschlagen.  
Die Einträge haben dann keine Angabe für die Sprache, weil die Angaben in der Quelle fehlt.

```sql
UPDATE place_label SET is_geonames_name = false;
```

```sql
INSERT INTO place_label (SELECT NULL, geonames_id, name, NULL, 0, 0, id, 1 FROM place where place_type_id = 1);
```